# Project : Instacard Market Basket

Instacart, an on-line grocery delivery company, released their first public dataset “The Instacart Online Grocery Shopping Dataset 2017” with the goal of predicting the previously purchased items a user will add in their next order. 

This anonymized dataset contains a sample of over 3 million grocery orders from more than 200,000 Instacart users : https://www.kaggle.com/c/instacart-market-basket-analysis/data

In this project, we will apply FPGrowth algorithm to find association rules from this dataset and build a recommender system in order to help Instacart improve customer shopping experience and increase sale.


In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
SparkContext.setSystemProperty('spark.hadoop.dfs.client.use.datanode.hostname', 'true')
sc=SparkContext(master='local', appName='New Spark Context')
sc

<SparkContext master=local appName=New Spark Context>

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('FPM_demo').getOrCreate()
spark

In [4]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

In [ ]:
# Loads data
data = spark.read.csv("D:/DS/instacart_online_grocery_shopping_2017_05_01/instacart_2017_05_01/order_products__train.csv", header=True, inferSchema=True)

In [11]:
data.count()

1384617

In [12]:
data.show(10)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
|       1|     13176|                6|        0|
|       1|     47209|                7|        0|
|       1|     22035|                8|        1|
|      36|     39612|                1|        0|
|      36|     19660|                2|        1|
+--------+----------+-----------------+---------+
only showing top 10 rows



In [13]:
data.createOrReplaceTempView("order_products_train")

In [14]:
products = spark.sql("select distinct product_id from order_products_train")
products.count()

39123

In [ ]:
rawData = spark.sql("select * from order_products_train")

baskets = rawData.groupBy('order_id').agg(collect_set('product_id').alias('items'))
baskets.show(5, False)

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                                                                                                                                                                                                                                                       |
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
fpg=FPGrowth(itemsCol='items', minSupport=0.003, minConfidence=0.003)
model=fpg.fit(baskets)

In [19]:
model.freqItemsets.show()

+-------+----+
|  items|freq|
+-------+----+
|[25146]|1054|
|[46676]| 993|
|[26940]| 730|
|[39993]| 708|
| [5025]|1445|
|[45200]| 444|
|[27325]| 441|
| [7781]|1342|
|[13629]| 772|
|[30776]| 855|
|[21376]| 753|
|[33120]| 834|
|[42625]| 658|
|[47630]| 796|
| [5785]|1535|
|[44560]| 451|
| [7644]| 398|
| [8859]| 950|
|[20734]| 420|
|[32864]| 914|
+-------+----+
only showing top 20 rows



In [ ]:
# Transform examines the input items against all the association rules and summarize the consequents as prediction
mostPopularItemInABasket = model.transform(baskets)

In [24]:
mostPopularItemInABasket.show(3, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 order_id   | 1                                                                                                                                                                                                                                                                                                                                                                                                        
 items      | [49683, 43633, 11109, 47209, 10246, 22035, 49302, 13176]                                                                                                                  

In [22]:
mostPopularItemInABasket.write.parquet("Chap9/MPIB.parquet", mode = "overwrite")

USE PRODUCT NAME INSTEAD OF PRODUCT ID

In [26]:
product_data = spark.read.csv("D:/Tochan/DS/BIG DATA/DATABASE/instacart_online_grocery_shopping_2017_05_01/instacart_2017_05_01/products.csv", header=True, inferSchema=True)

In [27]:
product_data.show(5, truncate=False)

+----------+-----------------------------------------------------------------+--------+-------------+
|product_id|product_name                                                     |aisle_id|department_id|
+----------+-----------------------------------------------------------------+--------+-------------+
|1         |Chocolate Sandwich Cookies                                       |61      |19           |
|2         |All-Seasons Salt                                                 |104     |13           |
|3         |Robust Golden Unsweetened Oolong Tea                             |94      |7            |
|4         |Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce|38      |1            |
|5         |Green Chile Anytime Sauce                                        |5       |13           |
+----------+-----------------------------------------------------------------+--------+-------------+
only showing top 5 rows



In [30]:
product_data.createOrReplaceTempView("products")

In [31]:
rawData_1 = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets_1 = rawData_1.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets_1.show(10, False)

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [32]:
fpGrowth_1 = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model_1 = fpGrowth_1.fit(baskets_1)

In [ ]:
# Display frequent itemsets
model_1.freqItemsets.show(truncate=False)

+----------------------------------------------------------------+----+
|items                                                           |freq|
+----------------------------------------------------------------+----+
|[Original Orange Juice]                                         |1054|
|[Total 0% Nonfat Greek Yogurt]                                  |993 |
|[Organic Cripps Pink Apples]                                    |730 |
|[Vine Ripe Tomatoes]                                            |708 |
|[Green Onions]                                                  |1445|
|[Organic Cauliflower]                                           |444 |
|[Organic Sticks Low Moisture Part Skim Mozzarella String Cheese]|1342|
|[Total 0% Raspberry Yogurt]                                     |441 |
|[Organic Snipped Green Beans]                                   |772 |
|[Organic Raw Kombucha Gingerade]                                |855 |
|[Organic Frozen Peas]                                          

In [34]:
mostPopularItemInABasket_1 = model_1.transform(baskets_1)

In [35]:
mostPopularItemInABasket_1.show(3, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

MAKE RECOMMENDATION

In [36]:
# List associationRules
model.associationRules.show()

+--------------+----------+--------------------+------------------+--------------------+
|    antecedent|consequent|          confidence|              lift|             support|
+--------------+----------+--------------------+------------------+--------------------+
|[30391, 13176]|   [21137]|   0.334384858044164| 4.027382305775355|0.003231485645039...|
|       [27156]|   [13176]|  0.2614213197969543|2.2158158881937067|0.003140028504142246|
|       [41950]|   [21137]| 0.19188337273443656| 2.311072650368339|0.003711635634750665|
|       [41950]|   [13176]|  0.2501970055161544| 2.120678223305498|0.004839607039151278|
|       [41950]|   [21903]| 0.16784869976359337| 2.250946448005041|0.003246728501855...|
|       [41950]|   [47209]| 0.19345941686367218|3.4805452663191505|0.003742121348383...|
|       [41950]|   [24852]| 0.16193853427895982|1.1346680094098065|0.003132407075734134|
|[47626, 24852]|   [26209]| 0.20945319740500462|  4.55530326177909|0.003444885640466...|
|[47626, 24852]|   [4

In [ ]:
# Filter out the rules that have an antecedent containing product_id
filtered_rules = model.associationRules.filter(array_contains(model.associationRules["antecedent"], 4605))
filtered_rules.show()

+----------+----------+-------------------+------------------+--------------------+
|antecedent|consequent|         confidence|              lift|             support|
+----------+----------+-------------------+------------------+--------------------+
|    [4605]|   [26209]| 0.1318447634237108|2.8674323825727948|0.003780228490423...|
|    [4605]|   [47766]|0.14221158958001062| 2.518482987880094|0.004077464198340053|
|    [4605]|   [47626]|0.19856459330143542|3.2026381957575953| 0.00569320702085985|
|    [4605]|   [21903]| 0.1446039340776183|1.9392209307430721|0.004146057054013063|
|    [4605]|   [24852]|  0.284688995215311|1.9947537313470969|0.008162549825088218|
+----------+----------+-------------------+------------------+--------------------+



In [ ]:
def get_associated_products(product_id, model, data, top_n=3):
    # Create a DataFrame
    association_rules = model.associationRules

    # # Filter out the rules that have an antecedent containing product_id
    filtered_rules = association_rules.filter(array_contains(association_rules["antecedent"], product_id))

    # Sort the rules by confidence in descending order and select the top_n products
    top_associated_products = (filtered_rules
                               .orderBy("confidence", ascending=False)
                               .limit(top_n)
                               .select(explode("consequent").alias("associated_product_id")))

    # Retrieve detailed product information from the DataFrame based on product_id
    associated_product_details = (top_associated_products
                                  .join(data, top_associated_products["associated_product_id"] == data["product_id"])
                                  .select("product_id", "product_name"))

    # Return a list of products that are bought together
    return associated_product_details.collect()

In [39]:
product_data.filter(product_data.product_id == 4605).show()

+----------+-------------+--------+-------------+
|product_id| product_name|aisle_id|department_id|
+----------+-------------+--------+-------------+
|      4605|Yellow Onions|      83|            4|
+----------+-------------+--------+-------------+



In [40]:
associated_products = get_associated_products(4605, model, product_data)

In [41]:
for product in associated_products:
    print(f"Product ID: {product['product_id']}, Product Name: {product['product_name']}")

Product ID: 21903, Product Name: Organic Baby Spinach
Product ID: 24852, Product Name: Banana
Product ID: 47626, Product Name: Large Lemon
